In [1]:
import numpy as np
import cvxopt
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import matplotlib.cm as cmx
import matplotlib.colors as colors

In [2]:
del_t = 0.1
v_init = 0.1
w_init = 0

class agent:
    def __init__(self,rad,start,goal,theta,vmax,wmax):
        self.rad = rad
        self.time = 0
        self.goal = goal
        self.theta = theta
        self.v = v_init
        self.w = w_init
        self.pos = start
        self.vmax = vmax
        self.wmax = wmax
        self.optfl = 0
    def __str__(self):
        return (str(self.time)+" "+str(self.theta)+" "+str(self.v)+" "+str(self.w)+" "+str(self.pos))
    def opt_traj(self):
        self.visualize_traj()
        cost_hist = []
        cost_hist.append(0)
        it = 0
        while (np.linalg.norm(self.pos-self.goal)>0.5):
#             self.visualize_traj()
            if (self.optfl==0):
                theta_new = self.theta + self.w*del_t
                xgi = self.pos[0] + self.v*del_t*np.cos(theta_new) - self.goal[0]
                ygi = self.pos[1] + self.v*del_t*np.sin(theta_new) - self.goal[1]
                p_mat = 2*np.array([[del_t**2,0],
                                  [0,(self.v**2)*(del_t**4)]])
                P = cvxopt.matrix(p_mat,tc='d')
                q_mat = np.array([2*del_t*(xgi*np.cos(theta_new)+ygi*np.sin(theta_new)),
                                 -2*del_t**2*self.v*(xgi*np.sin(theta_new)-ygi*np.cos(theta_new))])
    #             print (p_mat,q_mat)
                Q = cvxopt.matrix(q_mat,tc='d')
                g_mat = np.array([[1,0],[0,1],[0,-1]])
                h = cvxopt.matrix(np.array([self.vmax-self.v,self.wmax-self.w,self.wmax-self.w]),tc='d')
                g = cvxopt.matrix(g_mat,tc='d')
                sol = cvxopt.solvers.qp(P,Q,g,h,options={'show_progress': False})
                v_new = sol['x'][0]
                w_new = sol['x'][1]
                print ("v_new: ",v_new,"w_new: ",w_new)
                it+=1
                x_new = self.pos[0] + (self.v+v_new)*del_t*np.cos(self.theta+(self.w+w_new)*del_t)
                y_new = self.pos[1] + (self.v+v_new)*del_t*np.sin(self.theta+(self.w+w_new)*del_t)
                cost = np.linalg.norm(np.array([x_new,y_new])- self.goal)**2
                prev_cost = cost_hist[-1]
                print (it,cost)
                if (np.linalg.norm(np.array([v_new,w_new]))>0.1):
                    self.v += v_new 
                    self.w += w_new
                    print ("sending to optimiser: ",self.v,self.w)
                    cost_hist.append(cost)
                else:
                    self.optfl = 1
                    self.v += v_new 
                    self.w += w_new
                    self.theta = self.theta+self.w*del_t
                    print ("final v and w: ",self.v,self.w)
            else:
                x_new = self.pos[0] + self.v*del_t*np.cos(self.theta+self.w*del_t)
                y_new = self.pos[1] + self.v*del_t*np.sin(self.theta+self.w*del_t)
                self.pos = [x_new,y_new]
                print (self)
                self.time+=1
                self.visualize_traj()
                
             
    def visualize_traj(self):
        figure = plt.figure()
#         print (self.pos)
        print ("I wanna rotate by: ",self.theta)
        ax = figure.add_subplot(1,1,1)
        robot = matplotlib.patches.Rectangle(
            (self.pos[0]-self.rad*np.sqrt(2)*np.cos(self.theta+np.pi/4),self.pos[1]-self.rad*np.sqrt(2)*np.sin(self.theta+np.pi/4)),
            height = self.rad*2,
            width = self.rad*2,
            angle = self.theta*180/np.pi,
            edgecolor='black',
            linewidth=1.0,
            animated=True,
            alpha=1,
            zorder=2)
        ax.add_patch(robot)
        name = 'data/snap%s.png'%str(self.time)
        ax.plot([self.goal[0]], [self.goal[1]], '*', color="red", markersize =15,linewidth=3.0)
        ax.set_aspect('equal')
        ax.set_xlim(-10.0, 60.0)
        ax.set_ylim(-10.0, 60.0)
        ax.set_xlabel(r'$x (m)$')
        ax.set_ylabel(r'$y (m)$')
        ax.grid(True)
        plt.savefig(name, dpi = 200)
        plt.cla()
        plt.close(figure)
        return figure
    


In [ ]:
bot = agent(2,np.array([0,0]),np.array([50,50]),0,10,6)
bot.opt_traj()

I wanna rotate by:  0
v_new:  9.89999985815503 w_new:  5.999995387867064
1 4862.002205136303
sending to optimiser:  9.99999985815503 5.999995387867064
v_new:  1.4186817492527497e-07 w_new:  4.611732779502218e-06
2 4862.002191170254
final v and w:  10.000000000023205 5.999999999599844
0 0.5999999999599844 10.000000000023205 5.999999999599844 [0.3623577545521066, 0.9320390859403893]
I wanna rotate by:  0.5999999999599844
1 0.5999999999599844 10.000000000023205 5.999999999599844 [0.7247155091042132, 1.8640781718807786]
I wanna rotate by:  0.5999999999599844
2 0.5999999999599844 10.000000000023205 5.999999999599844 [1.0870732636563198, 2.7961172578211677]
I wanna rotate by:  0.5999999999599844
3 0.5999999999599844 10.000000000023205 5.999999999599844 [1.4494310182084265, 3.7281563437615572]
I wanna rotate by:  0.5999999999599844
4 0.5999999999599844 10.000000000023205 5.999999999599844 [1.811788772760533, 4.660195429701947]
I wanna rotate by:  0.5999999999599844
5 0.5999999999599844 10.000

59 0.5999999999599844 10.000000000023205 5.999999999599844 [21.74146527312642, 55.922345156423326]
I wanna rotate by:  0.5999999999599844
60 0.5999999999599844 10.000000000023205 5.999999999599844 [22.103823027678526, 56.854384242363714]
I wanna rotate by:  0.5999999999599844
61 0.5999999999599844 10.000000000023205 5.999999999599844 [22.466180782230634, 57.7864233283041]
I wanna rotate by:  0.5999999999599844
62 0.5999999999599844 10.000000000023205 5.999999999599844 [22.82853853678274, 58.71846241424449]
I wanna rotate by:  0.5999999999599844
63 0.5999999999599844 10.000000000023205 5.999999999599844 [23.190896291334848, 59.65050150018488]
I wanna rotate by:  0.5999999999599844
64 0.5999999999599844 10.000000000023205 5.999999999599844 [23.553254045886955, 60.58254058612527]
I wanna rotate by:  0.5999999999599844
65 0.5999999999599844 10.000000000023205 5.999999999599844 [23.915611800439063, 61.51457967206566]
I wanna rotate by:  0.5999999999599844
66 0.5999999999599844 10.0000000000

119 0.5999999999599844 10.000000000023205 5.999999999599844 [43.48293054625285, 111.84469031284665]
I wanna rotate by:  0.5999999999599844
120 0.5999999999599844 10.000000000023205 5.999999999599844 [43.84528830080496, 112.77672939878704]
I wanna rotate by:  0.5999999999599844
121 0.5999999999599844 10.000000000023205 5.999999999599844 [44.20764605535707, 113.70876848472743]
I wanna rotate by:  0.5999999999599844
122 0.5999999999599844 10.000000000023205 5.999999999599844 [44.570003809909174, 114.64080757066782]
I wanna rotate by:  0.5999999999599844
123 0.5999999999599844 10.000000000023205 5.999999999599844 [44.93236156446128, 115.5728466566082]
I wanna rotate by:  0.5999999999599844
124 0.5999999999599844 10.000000000023205 5.999999999599844 [45.29471931901339, 116.5048857425486]
I wanna rotate by:  0.5999999999599844
125 0.5999999999599844 10.000000000023205 5.999999999599844 [45.657077073565496, 117.43692482848898]
I wanna rotate by:  0.5999999999599844
126 0.5999999999599844 10.0